<div>
<img src="figures/svtLogo.png"/>
</div>

<center><h1>Mathematical Optimization for Engineers</h1></center>
<center><h2>Bonus exercise 1</h2></center>

In this exercise, you will <u>implement and solve</u> the optimization problem that we formulated in Lab 01.<br>
<br>
You should use the template below, for submission. 
<br>
<br>
- As a rule of thumb, you should <b>only add code</b> to this file and <b>not delete or change anything in the template</b>.
<br>
<br>
- To see where you can add code, go ahead and do a search (ctrl+f) for '# YOUR CODE HERE'
<br>
<br>
- Once you have added your code at these spots, delete the subsequent 'raise NotImplementedError()'. These exist only to remind you to add your own code.
<br>
<br>
- Most importantly, have fun while programing :)!

Complete the following tasks as you program:

1. Use the <i>linprog</i> solver from the <i>scipy</i> package to solve the problem.
<br>
<br>
2. Is the demand penalty active in the optimal solution? (recall that you consider two cases)

In [ ]:
# Please keep in mind the units of all optimization variables while programing

# I1, InFlow rate of Turbine_1, Unit kg/h
# I2, InFlow rate of Turbine_2, Unit kg/h
# HE1, Medium Pressure Steam outFlow rate from turbine 1, Unit kg/h
# HE2, Medium Pressure Steam outFlow rate from turbine 2, Unit kg/h
# LE1, Low-Pressure Steam outFlow rate from turbine 1, Unit kg/h
# LE2, Low-Pressure Steam outFlow rate from turbine 2, Unit kg/h
# C, Condensate Flow Rate, Unit kg/h
# BF1, by-pass flow rate, Unit kg/h
# BF2, by-pass flow rate, Unit kg/h
# HPS, High pressure steam flow, Unit kg/h
# MPS, Medium pressure steam flow, Unit kg/h
# LPS, Low pressure steam flow, Unit kg/h
# P1, Power genererated by turbine 1, Unit kW
# P2, Power genererated by turbine 2, Unit kW
# PP, Total Power produced by both the turbines, Unit kW
# EP, Power purchased, Unit kW
# Power, Total power delivered by the system,Unit kW
# Fuel, Fuel consumed by the system, Unit kJ/h

Tip: every solver expects input in its own unique format. We should inform ourselves of this format by looking at the documentation of the solver <u>before</u> we start to implement our problem.

[https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html](URL 'Click here for linprog documentation')

`linprog(c, Aub, bub, Aeq, beq, bounds, method='interior-point', options, x0)`


\begin{align}
\min_\mathbf x \ & \mathbf c^T \mathbf x \\
        \mbox{s. t.} \; \ & \mathbf A_{ub} \mathbf x \leq \mathbf b_{ub},\\
        & \mathbf A_{eq} \mathbf x = \mathbf b_{eq},\\
        & \mathbf l \leq \mathbf x \leq \mathbf u ,
\end{align}


where $\mathbf x$ is a vector of decision variables; $\mathbf c$,
$\mathbf b_{ub}$, $\mathbf b_{eq}$, $\mathbf l$, and $\mathbf u$ are vectors; and
$\mathbf A_{ub}$ and $\mathbf A_{eq}$ are matrices.

In [241]:
import numpy as np
from scipy.optimize import linprog

In [242]:
# dimensions of problem 
numVars = 18
numEqC = 10
numInEqC = 2

# define indicies for variables
I1, I2, HE1, HE2, LE1, LE2, C, BF1, BF2, HPS, MPS, LPS, P1, P2, PP, EP, Power, Fuel = (i for i in range(numVars))

# prices from Table 2
fuel_cost_coeff = 1.5*10 ** (-6) # Unit Euro/kJ
condensate_loss_coeff = 0.008    # Unit Euro/kg
produced_power_coeff = 0.02      # Unit Euro/kWh
purchased_power_coeff = 0.05     # Unit Euro/kWh
demand_penalty = 0.001           # Unit Euro/kWh

# enthalpies from Table 4
Enthalpy_of_HPS = 3163           # Unit kJ/kg
enthalpy_mps = 2949              # Unit kJ/kg
enthalpy_lps = 2911              # Unit kJ/kg 
enthalpy_condensate = 449        # Unit kJ/kg

# data from Table 5
Evaporator_Efficiency = 0.75     # Unit -
basic_power = 12000              # Unit kW

In [243]:
# coefficients in the objective function (10 points)
# input: nothing
# output: numpy array of coefficients in the objective function
#I1, I2, HE1, HE2, LE1, LE2, C, BF1, BF2, HPS, MPS, LPS, P1, P2, PP, EP, Power, Fuel
def objCoeffs():
    # initialize
    # c = np.zeros(...)
    # c[PP]=0.02 
    # ...
    # we will set c[EP] later, depending on the case
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    c = np.zeros(18)
    c[C] = condensate_loss_coeff
    c[PP] = produced_power_coeff
    c[Fuel] = fuel_cost_coeff
    return c


In [244]:
objCoeffs()

array([0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 8.0e-03,
       0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00,
       2.0e-02, 0.0e+00, 0.0e+00, 1.5e-06])

In [245]:
# please leave this cell as it is

In [246]:
# inequality constraints (10 points)
# input: nothing
# output: numpy arrays Aub and bub
def IneqCons():
    
    # initialize
    # Aub = np.zeros...
    # bub = np.zeros...

    # I1-HE1 <= 60000 
    # Aub[0,I1]=1
    # Aub[0,HE1]=-1
    # bub[0]=60000

    # P1+P2+EP >= 24500
    # ...

    # YOUR CODE HERE
    #raise NotImplementedError()
    Aub = np.zeros([2,18])
    bub = np.zeros(2)
    Aub[:,I1] = [1,0]
    Aub[:,HE1] = [-1,0]
    Aub[:,P1] = [0,-1]
    Aub[:,P2] = [0,-1]
    Aub[:,EP] = [0,-1]
    bub[0] = 60000
    bub[1] = -24500
    return Aub, bub

In [247]:
IneqCons()

(array([[ 1.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -1.,
         -1.,  0., -1.,  0.,  0.]]),
 array([ 60000., -24500.]))

In [248]:
# please leave this cell as it is

In [249]:
# equality constraints (10 points)
# input: nothing
# output: numpy arrays Aeq and beq

#HPS, I2, BF1, LE1, HE2, BF2, MPS, LPS = sim.symbols('HPS I2 BF1 LE1 HE2 BF2 MPS LPS')
def EqCons():
    
    # Aeq = np.zeros ...
    # beq = np.zeros ...

    # Enthalpy_of_HPS*HPS=Evaporator_Efficiency*Fuel
    # Aeq[0,Fuel]=Evaporator_Efficiency
    # Aeq[0,HPS]=-Enthalpy_of_HPS

    # HPS = I1+I2+BF1
    # ...
    
    # I1 = HE1+LE1+C
    # ...
    
    # ...
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    Aeq = np.zeros([10,18])
    beq = np.zeros(10)
    #Equation MB1
    Aeq[0,I1] = 1
    Aeq[0,I2] = 1
    Aeq[0,BF1] = 1
    Aeq[0,HPS] = -1
    
    #Equation MB2
    Aeq[1,I1] = -1
    Aeq[1,HE1] = 1
    Aeq[1,LE1] = 1
    Aeq[1,C] = 1
    
    #Equation MB3
    Aeq[2,I2] = -1
    Aeq[2,HE2] = 1
    Aeq[2,LE2] = 1
    
    #Equation MB4
    Aeq[3,HE1] = 1
    Aeq[3,HE2] = 1
    Aeq[3,BF1] = 1
    Aeq[3,BF2] = -1
    Aeq[3,MPS] = -1
    
    #Equation MB5
    Aeq[4,LE1] = 1
    Aeq[4,LE2] = 1
    Aeq[4,BF2] = 1
    Aeq[4,LPS] = -1
    
    #Equation EB1
    Aeq[5,I1] = -3163
    Aeq[5,HE1] = 2949
    Aeq[5,LE1] = 2911
    Aeq[5,C] = 449
    Aeq[5,P1] = 3600
    
    #Equation EB2
    Aeq[6,I2] = -3163
    Aeq[6,HE2] = 2949
    Aeq[6,LE2] = 2911
    Aeq[6,P2] = 3600
    
    #Equation EB3
    Aeq[7,P1] = 1
    Aeq[7,P2] = 1
    Aeq[7,PP] = -1
    
    #Equation EB4
    Aeq[8,PP] = 1
    Aeq[8,EP] = 1
    Aeq[8, Power] = -1
    
    #Equation EB5
    Aeq[9,HPS] = -Enthalpy_of_HPS
    Aeq[9,Fuel] = Evaporator_Efficiency
    
    return Aeq, beq

In [250]:
EqCons()

(array([[ 1.000e+00,  1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  1.000e+00,  0.000e+00, -1.000e+00,
          0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  0.000e+00],
        [-1.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,  1.000e+00,
          0.000e+00,  1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  0.000e+00],
        [ 0.000e+00, -1.000e+00,  0.000e+00,  1.000e+00,  0.000e+00,
          1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  0.000e+00],
        [ 0.000e+00,  0.000e+00,  1.000e+00,  1.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  1.000e+00, -1.000e+00,  0.000e+00,
         -1.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
          0.000e+00,  0.000e+00,  

In [251]:
# please leave this cell as it is

In [252]:
# lower and upper bounds for variables (10 points)
# input: nothing
# output: numpy arrays l and u

def bounds():
    
    # l = np.zeros...
    # u = np.ones...

    # l[P1]=2500
    # ...

    # u[P1]=6250
    # ...

    # YOUR CODE HERE
    #raise NotImplementedError()
    l = np.zeros(18)
    u = np.ones(18)
    #Lower bounds:
    l[I1] = 0
    l[I2] = 0
    l[HE1] = 0
    l[HE2] = 0
    l[LE1] = 0
    l[LE2] = 0
    l[C] = 0
    l[BF1] = 0
    l[BF2] = 0
    l[HPS] = 0
    l[MPS] = 123000
    l[LPS] = 45000
    l[P1] = 2500
    l[P2] = 3000
    l[PP] = 0
    l[EP] = 0
    l[Power] = 0
    l[Fuel] = 0
    
    #Upper Bounds:
    u[I1] = 87000
    u[I2] = 110000
    u[HE1] = None
    u[HE2] = None
    u[LE1] = None
    u[LE2] = 64000
    u[C] = 28000
    u[BF1] = None
    u[BF2] = None
    u[HPS] = None
    u[MPS] = None
    u[LPS] = None
    u[P1] = 6250
    u[P2] = 9000
    u[PP] = None
    u[EP] = 1
    u[Power] = None
    u[Fuel] = None
    
    return l,u

In [253]:
bounds()

(array([     0.,      0.,      0.,      0.,      0.,      0.,      0.,
             0.,      0.,      0., 123000.,  45000.,   2500.,   3000.,
             0.,      0.,      0.,      0.]),
 array([8.70e+04, 1.10e+05,      nan,      nan,      nan, 6.40e+04,
        2.80e+04,      nan,      nan,      nan,      nan,      nan,
        6.25e+03, 9.00e+03,      nan, 1.00e+00,      nan,      nan]))

In [254]:
# please leave this cell as it is

In [255]:
# demand penalty not active (25 points)
# EP >= 12 MW
# input: nothing
# output: cost, EPval (value of additional power purchased) and optimal operating point (i.e. the values of I1, I2, HE1, HE2, LE1, LE2, C, BF1, BF2, HPS, MPS, LPS, P1, P2, PP, EP, Power, Fuel)

def notActive():
    c = objCoeffs()
    Aub, bub = IneqCons()
    Aeq, beq = EqCons()
    l, u = bounds()
    # collect bounds in the correct format for using linprog        
    # bnds = ...
    # demand penalty is not active in this case
    # c[EP] = ...
    # bnds[EP] = ...

    # YOUR CODE HERE
    #raise NotImplementedError()
    c[EP] = purchased_power_coeff
    l[EP] = 12000
    u[EP] = None
    l = np.transpose(l)
    u = np.transpose(u)
    limits = np.column_stack((l,u))
    bnds = list(map(tuple, limits))
    res = linprog(c, Aub, bub, Aeq, beq, bnds, method='interior-point')
    cost = res.fun
    EPval = res.x[EP]
    OptPoint = res.x       
    return cost, EPval, OptPoint

In [256]:
notActive()

(1951.5020425237374,
 12000.000032880318,
 array([7.08523033e+04, 9.98514550e+04, 5.97556492e+04, 6.32443513e+04,
        8.39289576e+03, 3.66071037e+04, 2.70375826e+03, 6.79553635e-05,
        5.47117768e-04, 1.70703758e+05, 1.23000000e+05, 4.50000000e+04,
        6.17797738e+03, 6.32202259e+03, 1.25000000e+04, 1.20000000e+04,
        2.45000000e+04, 7.19914650e+08]))

In [257]:
# please leave this cell as it is

In [258]:
# demand penalty active (25 points)
# EP < 12 MW
# input: nothing
# output: cost, EPval (value of additional power purchased) and optimal operating point (i.e. the values of I1, I2, HE1, HE2, LE1, LE2, C, BF1, BF2, HPS, MPS, LPS, P1, P2, PP, EP, Power, Fuel)
demand_penatly = 0.001
def active():
    
    c = objCoeffs()
    Aub, bub = IneqCons()
    Aeq, beq = EqCons()
    l, u = bounds()
        
    # collect bounds in the correct format for using linprog        
    # bnds = ...
    
    # demand penalty is active in this case
    # c[EP] = ...
    # bnds[EP] = ...
    
    # YOUR CODE HERE
    #raise NotImplementedError() 
    c[EP] = purchased_power_coeff-demand_penalty
    l[EP] = 0
    u[EP] = 12000
    l = np.transpose(l)
    u = np.transpose(u)
    limits = np.column_stack((l,u))
    bnds = list(map(tuple, limits))
    res = linprog(c, Aub, bub, Aeq, beq, bnds, method='interior-point')
    cost = res.fun+(demand_penatly*12000)
    EPval = res.x[EP]
    OptPoint = res.x      
    return cost, EPval, OptPoint

In [259]:
active()

(1943.8653083236452,
 11236.111261543376,
 array([6.17170220e+04, 1.10000000e+05, 5.79999932e+04, 6.50000068e+04,
        6.08240746e-03, 4.49999932e+04, 3.71702274e+03, 2.86628604e-03,
        1.70517396e-03, 1.71717025e+05, 1.23000001e+05, 4.50000009e+04,
        6.24999994e+03, 7.01388882e+03, 1.32638888e+04, 1.12361113e+04,
        2.45000000e+04, 7.24187933e+08]))

In [260]:
# please leave this cell as it is

Is the demand penalty active in the optimal solution? (Type your answer in the cell below, 10 points)

Yes, the demand penalty is active in the optimal solution as the cost is minimum in that case